from https://www.kaggle.com/dinodelao/fastai-v2-gpu-for-chinese-mnist-prediction

In [ ]:
import fastai
from fastai.vision.all import *
from fastai.vision.widgets import *
import pandas as pd
import os

In [ ]:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
path = Path("../input/chinese-mnist/data/data")

Path.BASE_PATH = path

path.ls()

creating a dataframe

In [ ]:
df = pd.read_csv("../input/chinese-mnist/chinese_mnist.csv")
df.head()

In [ ]:
# We can create a new column in Pandas and use our existing columns to concatenate our file names.

df['fname'] = ("input_" + df['suite_id'].astype(str) 
               + "_" 
               + df['sample_id'].astype(str) 
               + "_" 
               + df['code'].astype(str) 
               + ".jpg")
df.head()

drfine functions to get the label items

In [ ]:
def get_x(r): return path/r['fname']

#.astype() and .split() method were added to contain each label
def get_y(r): return r['value'].astype(str).split(" ")

In [ ]:
dblock = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   splitter = RandomSplitter(seed=42),
                   get_x = get_x,
                   get_y = get_y,
                   item_tfms = RandomResizedCrop(128, min_scale=0.35),
                  )
dls = dblock.dataloaders(df)

In [ ]:
len(dls.train), len(dls.valid)

In [ ]:
dls.show_batch()

In [ ]:
dls.train_ds

In [ ]:
dls.valid_ds

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=partial(accuracy_multi,thresh=0.2))
learn.fine_tune(6)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_top_losses(5, nrows=1)


In [ ]:
interp.most_confused(5)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=partial(accuracy_multi, thresh=0.2))
learn.lr_find()

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=partial(accuracy_multi, thresh=0.2))
learn.fine_tune(6, base_lr=1e-2)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(6, 1e-4)

In [ ]:
learn.show_results()